In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from math import sqrt
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
train=pd.read_csv(r'train.csv')
test=pd.read_csv(r'test.csv')

x_train = train
x_train['Date'] = pd.to_datetime(x_train['Date'], errors='coerce')
x_train


# x_train['Season'] = x_train['Date'].dt.month
# x_train['Season']=x_train['Season'].replace([4,5,6,7,8,9],1)
# x_train['Season']=x_train['Season'].replace([1,2,3,10,11,12],0)
# x_train['Weekend'] = x_train['Date'].dt.dayofweek
# x_train['Weekend']=x_train['Weekend'].replace([0,1,2,3,4,5,6],[0,0,0,0,1,1,0])
# x_train.drop(['Date'],inplace=True,axis=1)

x_train_A = x_train[x_train['Procedure']=='A']
x_train_B = x_train[x_train['Procedure']=='B']
x_train_C = x_train[x_train['Procedure']=='C']

# P_1 = [1 if x is 'A' else 0 for x in x_train['Procedure']]
# P_2 = [1 if x is 'B' else 0 for x in x_train['Procedure']]
# P_3 = [1 if x is 'C' else 0 for x in x_train['Procedure']]
# x_train['P_1'] = P_1
# x_train['P_2'] = P_2
# x_train['P_3'] = P_3
y_train_A = x_train_A['n_Procedure']
y_train_B = x_train_B['n_Procedure']
y_train_C = x_train_B['n_Procedure']
x_train_A.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)
x_train_B.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)
x_train_C.drop(['Procedure','Date','n_Procedure'],inplace=True,axis=1)

# scaler = StandardScaler()
# x_train_A=scaler.fit_transform(x_train_A)

# x_train_B=scaler.fit_transform(x_train_B)
# x_train_C=scaler.fit_transform(x_train_C)
x_train_A=np.array(x_train_A)
x_train_B=np.array(x_train_B)
x_train_C=np.array(x_train_C)
print(x_train_B.shape)
x_train_A = np.reshape(x_train_A, (x_train_A.shape[0], 1, x_train_A.shape[1]))
x_train_B = np.reshape(x_train_B, (x_train_B.shape[0], 1, x_train_B.shape[1]))
x_train_C = np.reshape(x_train_C, (x_train_C.shape[0], 1, x_train_C.shape[1]))

print(np.shape(x_train_A))

k = 1
x_A,y_A = x_train_A[:-k],y_train_A[:-k]
x_t_A,y_t_A = x_train_A[-k:],y_train_A[-k:]
print(x_A.shape)


x_B,y_B = x_train_B[:-k],y_train_B[:-k]
x_t_B,y_t_B = x_train_B[-k:],y_train_B[-k:]
print(x_B.shape)


x_C,y_C = x_train_C[:-k],y_train_C[:-k]
x_t_C,y_t_C = x_train_C[-k:],y_train_C[-k:]
print(x_C.shape)

(326, 2)
(326, 1, 2)
(325, 1, 2)
(325, 1, 2)
(325, 1, 2)


/home/shobhit/Documents/interiit/env/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
# mu = scaler.mu

# std = np.sqrt(scaler.var_)
# mu = scaler.mean_
# std,mu

In [22]:
import math
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM,Bidirectional
def train_pred_eval_model(x_train_scaled, \
                          y_train_scaled, \
                          x_test_scaled, \
                          y_test, \
                          lstm_units, \
                          dropout_prob, \
                          optimizer, \
                          epochs, \
                          batch_size):
    '''
    Train model, do prediction, scale back to original range and do 
    evaluation
    Use LSTM here.
    Returns rmse, mape and predicted values
    Inputs
        x_train_scaled  : e.g. x_train_scaled.shape=(451, 9, 1). 
                          Here we are using the past 9 values to  
                          predict the next value
        y_train_scaled  : e.g. y_train_scaled.shape=(451, 1)
        x_test_scaled   : use this to do predictions 
        y_test          : actual value of the predictions
        mu_test_list    : list of the means. Same length as 
                          x_test_scaled and y_test
        std_test_list   : list of the std devs. Same length as 
                          x_test_scaled and y_test
        lstm_units      : dimensionality of the output space
        dropout_prob    : fraction of the units to drop for the 
                          linear transformation of the inputs
        optimizer       : optimizer for model.compile()
        epochs          : epochs for model.fit()
        batch_size      : batch size for model.fit()
    Outputs
        rmse            : root mean square error
        mape            : mean absolute percentage error
        est             : predictions
    '''
    # Create the LSTM network
    model = Sequential()
#     print(x_train_scaled.shape,y_train_scaled.shape,(x_train_scaled.shape[1],1))
    model.add((LSTM(units=lstm_units,
                   return_sequences=True, 
                   input_shape=(1,x_train_scaled.shape[2]))))
    # Add dropput with a probability of 0.5
    model.add(Dropout(dropout_prob)) 
    model.add(Bidirectional(LSTM(units=lstm_units)))
    # Add dropput with a probability of 0.5
    model.add(Dropout(dropout_prob)) 
    model.add(Dense(1))
    # Compile and fit the LSTM network
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(x_train_scaled, y_train_scaled, epochs=epochs,   
              batch_size=batch_size, verbose=1)
    est=[]
#     # Do prediction
    est = model.predict(x_test_scaled)
#     est = (est_scaled * np.array(std_test_list).reshape(-1,1)) +  np.array(mu_test_list).reshape(-1,1)
    
    # Calculate RMSE and MAPE
    rmse = math.sqrt(mean_squared_error(y_test, est))
    print(rmse)
    print(np.exp(-rmse))
    return rmse, est,model

In [23]:
 val_A=train_pred_eval_model(x_A,y_A, x_A,y_A,5,0.5,'adam',1000,10)                            

Epoch 1/1000
325/325 [==============================] - 18s 55ms/step - loss: 33.6359


/home/shobhit/Documents/interiit/env/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.341414). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 2/1000
325/325 [==============================] - 0s 1ms/step - loss: 32.4390
Epoch 3/1000
325/325 [==============================] - 0s 246us/step - loss: 30.8012
Epoch 4/1000
325/325 [==============================] - 0s 261us/step - loss: 27.8398
Epoch 5/1000
325/325 [==============================] - 0s 262us/step - loss: 24.6917
Epoch 6/1000
325/325 [==============================] - 0s 307us/step - loss: 21.7987
Epoch 7/1000
325/325 [==============================] - 0s 544us/step - loss: 19.9634
Epoch 8/1000
325/325 [==============================] - 0s 315us/step - loss: 16.8160
Epoch 9/1000
325/325 [==============================] - 0s 311us/step - loss: 14.8179
Epoch 10/1000
325/325 [==============================] - 0s 315us/step - loss: 14.0887
Epoch 11/1000
325/325 [==============================] - 0s 329us/step - loss: 12.4252
Epoch 12/1000
325/325 [==============================] - ETA: 0s - loss: 10.08 - 0s 285us/step - loss: 10.0322
Epoch 13/1000
325/325 [======

325/325 [==============================] - 0s 303us/step - loss: 5.9993
Epoch 191/1000
325/325 [==============================] - 0s 373us/step - loss: 5.5080
Epoch 192/1000
325/325 [==============================] - 0s 304us/step - loss: 6.0140
Epoch 193/1000
325/325 [==============================] - 0s 344us/step - loss: 5.7859
Epoch 194/1000
325/325 [==============================] - 0s 310us/step - loss: 5.6954
Epoch 195/1000
325/325 [==============================] - 0s 325us/step - loss: 6.0725
Epoch 196/1000
325/325 [==============================] - 0s 301us/step - loss: 5.6715
Epoch 197/1000
325/325 [==============================] - 0s 312us/step - loss: 6.1373
Epoch 198/1000
325/325 [==============================] - 0s 290us/step - loss: 5.4977
Epoch 199/1000
325/325 [==============================] - 0s 296us/step - loss: 5.8017
Epoch 200/1000
325/325 [==============================] - 0s 319us/step - loss: 5.9382
Epoch 201/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 295us/step - loss: 4.9332
Epoch 379/1000
325/325 [==============================] - 0s 291us/step - loss: 5.2477
Epoch 380/1000
325/325 [==============================] - 0s 281us/step - loss: 5.3888
Epoch 381/1000
325/325 [==============================] - 0s 279us/step - loss: 5.0240
Epoch 382/1000
325/325 [==============================] - 0s 289us/step - loss: 5.0068
Epoch 383/1000
325/325 [==============================] - 0s 285us/step - loss: 5.1984
Epoch 384/1000
325/325 [==============================] - 0s 282us/step - loss: 5.2108
Epoch 385/1000
325/325 [==============================] - 0s 277us/step - loss: 5.0362
Epoch 386/1000
325/325 [==============================] - 0s 286us/step - loss: 5.2264
Epoch 387/1000
325/325 [==============================] - 0s 291us/step - loss: 4.9484
Epoch 388/1000
325/325 [==============================] - 0s 281us/step - loss: 5.3132
Epoch 389/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 285us/step - loss: 4.9088
Epoch 567/1000
325/325 [==============================] - 0s 296us/step - loss: 5.0257
Epoch 568/1000
325/325 [==============================] - 0s 284us/step - loss: 4.8528
Epoch 569/1000
325/325 [==============================] - 0s 285us/step - loss: 4.9139
Epoch 570/1000
325/325 [==============================] - 0s 286us/step - loss: 4.9116
Epoch 571/1000
325/325 [==============================] - 0s 279us/step - loss: 4.9808
Epoch 572/1000
325/325 [==============================] - 0s 292us/step - loss: 4.8735
Epoch 573/1000
325/325 [==============================] - 0s 279us/step - loss: 4.9355
Epoch 574/1000
325/325 [==============================] - 0s 283us/step - loss: 4.9016
Epoch 575/1000
325/325 [==============================] - 0s 278us/step - loss: 4.9727
Epoch 576/1000
325/325 [==============================] - 0s 280us/step - loss: 4.8822
Epoch 577/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 273us/step - loss: 4.8768
Epoch 755/1000
325/325 [==============================] - 0s 288us/step - loss: 4.8396
Epoch 756/1000
325/325 [==============================] - 0s 275us/step - loss: 4.8012
Epoch 757/1000
325/325 [==============================] - 0s 284us/step - loss: 4.8268
Epoch 758/1000
325/325 [==============================] - 0s 275us/step - loss: 4.9217
Epoch 759/1000
325/325 [==============================] - 0s 289us/step - loss: 4.9086
Epoch 760/1000
325/325 [==============================] - 0s 283us/step - loss: 4.7918
Epoch 761/1000
325/325 [==============================] - 0s 277us/step - loss: 4.8246
Epoch 762/1000
325/325 [==============================] - 0s 286us/step - loss: 4.9061
Epoch 763/1000
325/325 [==============================] - 0s 286us/step - loss: 4.8310
Epoch 764/1000
325/325 [==============================] - 0s 291us/step - loss: 4.8600
Epoch 765/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 258us/step - loss: 4.8549
Epoch 943/1000
325/325 [==============================] - 0s 262us/step - loss: 4.8354
Epoch 944/1000
325/325 [==============================] - 0s 265us/step - loss: 4.8983
Epoch 945/1000
325/325 [==============================] - 0s 259us/step - loss: 4.8917
Epoch 946/1000
325/325 [==============================] - 0s 262us/step - loss: 4.7841
Epoch 947/1000
325/325 [==============================] - 0s 265us/step - loss: 4.9018
Epoch 948/1000
325/325 [==============================] - 0s 260us/step - loss: 4.8475
Epoch 949/1000
325/325 [==============================] - 0s 257us/step - loss: 4.8431
Epoch 950/1000
325/325 [==============================] - 0s 259us/step - loss: 4.8900
Epoch 951/1000
325/325 [==============================] - 0s 255us/step - loss: 4.8423
Epoch 952/1000
325/325 [==============================] - 0s 268us/step - loss: 4.8763
Epoch 953/1000
325/325 [==============================] - 

In [24]:
pred_A = val_A[1]
pred1_A = np.round(pred_A)

In [25]:
pred1_A

array([[5.],
       [6.],
       [5.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],

In [26]:
# z_A=y_t_A
# r = math.sqrt(mean_squared_error(z_A,pred1_A))
# r

In [27]:
# print(np.exp(-r))

In [29]:
val_B=train_pred_eval_model(x_B,y_B,x_B,y_B,3,0.7,'adam',1000,10)


Epoch 1/1000
325/325 [==============================] - 21s 66ms/step - loss: 10.3344
Epoch 2/1000


/home/shobhit/Documents/interiit/env/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.156445). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


325/325 [==============================] - 1s 2ms/step - loss: 10.0238
Epoch 3/1000
325/325 [==============================] - 0s 223us/step - loss: 9.7657
Epoch 4/1000
325/325 [==============================] - 0s 297us/step - loss: 9.3483
Epoch 5/1000
325/325 [==============================] - 0s 299us/step - loss: 8.9707
Epoch 6/1000
325/325 [==============================] - 0s 291us/step - loss: 8.7445
Epoch 7/1000
325/325 [==============================] - 0s 328us/step - loss: 8.4845
Epoch 8/1000
325/325 [==============================] - 0s 319us/step - loss: 8.1871
Epoch 9/1000
325/325 [==============================] - 0s 291us/step - loss: 7.9425
Epoch 10/1000
325/325 [==============================] - 0s 300us/step - loss: 7.4458
Epoch 11/1000
325/325 [==============================] - 0s 294us/step - loss: 7.1078
Epoch 12/1000
325/325 [==============================] - 0s 279us/step - loss: 6.6560
Epoch 13/1000
325/325 [==============================] - 0s 275us/step - los

325/325 [==============================] - 0s 308us/step - loss: 2.9761
Epoch 192/1000
325/325 [==============================] - 0s 289us/step - loss: 3.0344
Epoch 193/1000
325/325 [==============================] - 0s 275us/step - loss: 3.0314
Epoch 194/1000
325/325 [==============================] - 0s 278us/step - loss: 3.0987
Epoch 195/1000
325/325 [==============================] - 0s 278us/step - loss: 2.9820
Epoch 196/1000
325/325 [==============================] - 0s 282us/step - loss: 2.9416
Epoch 197/1000
325/325 [==============================] - 0s 282us/step - loss: 2.9810
Epoch 198/1000
325/325 [==============================] - 0s 288us/step - loss: 2.9983
Epoch 199/1000
325/325 [==============================] - 0s 291us/step - loss: 2.9647
Epoch 200/1000
325/325 [==============================] - 0s 283us/step - loss: 3.0781
Epoch 201/1000
325/325 [==============================] - 0s 277us/step - loss: 2.9989
Epoch 202/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 251us/step - loss: 2.9657
Epoch 380/1000
325/325 [==============================] - 0s 281us/step - loss: 2.9692
Epoch 381/1000
325/325 [==============================] - 0s 256us/step - loss: 2.9547
Epoch 382/1000
325/325 [==============================] - 0s 245us/step - loss: 2.9807
Epoch 383/1000
325/325 [==============================] - 0s 244us/step - loss: 2.9611
Epoch 384/1000
325/325 [==============================] - 0s 244us/step - loss: 2.9764
Epoch 385/1000
325/325 [==============================] - 0s 244us/step - loss: 2.9676
Epoch 386/1000
325/325 [==============================] - 0s 249us/step - loss: 2.9545
Epoch 387/1000
325/325 [==============================] - 0s 246us/step - loss: 2.9603
Epoch 388/1000
325/325 [==============================] - 0s 274us/step - loss: 2.9654
Epoch 389/1000
325/325 [==============================] - 0s 246us/step - loss: 2.9589
Epoch 390/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 281us/step - loss: 2.9648
Epoch 474/1000
325/325 [==============================] - 0s 297us/step - loss: 2.9718
Epoch 475/1000
325/325 [==============================] - 0s 301us/step - loss: 2.9758
Epoch 476/1000
325/325 [==============================] - 0s 280us/step - loss: 2.9703
Epoch 477/1000
325/325 [==============================] - 0s 262us/step - loss: 2.9640
Epoch 478/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9657
Epoch 479/1000
325/325 [==============================] - 0s 251us/step - loss: 2.9659
Epoch 480/1000
325/325 [==============================] - 0s 256us/step - loss: 2.9692
Epoch 481/1000
325/325 [==============================] - 0s 249us/step - loss: 2.9646
Epoch 482/1000
325/325 [==============================] - 0s 248us/step - loss: 2.9631
Epoch 483/1000
325/325 [==============================] - 0s 281us/step - loss: 2.9684
Epoch 484/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 253us/step - loss: 2.9616
Epoch 662/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9645
Epoch 663/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9629
Epoch 664/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9583
Epoch 665/1000
325/325 [==============================] - 0s 248us/step - loss: 2.9742
Epoch 666/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9644
Epoch 667/1000
325/325 [==============================] - 0s 246us/step - loss: 2.9611
Epoch 668/1000
325/325 [==============================] - 0s 245us/step - loss: 2.9627
Epoch 669/1000
325/325 [==============================] - 0s 249us/step - loss: 2.9605
Epoch 670/1000
325/325 [==============================] - 0s 251us/step - loss: 2.9642
Epoch 671/1000
325/325 [==============================] - 0s 246us/step - loss: 2.9737
Epoch 672/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 271us/step - loss: 2.9685
Epoch 850/1000
325/325 [==============================] - 0s 301us/step - loss: 2.9572
Epoch 851/1000
325/325 [==============================] - 0s 283us/step - loss: 2.9510
Epoch 852/1000
325/325 [==============================] - 0s 272us/step - loss: 2.9680
Epoch 853/1000
325/325 [==============================] - 0s 263us/step - loss: 2.9728
Epoch 854/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9730
Epoch 855/1000
325/325 [==============================] - 0s 272us/step - loss: 2.9460
Epoch 856/1000
325/325 [==============================] - 0s 284us/step - loss: 2.9621
Epoch 857/1000
325/325 [==============================] - 0s 285us/step - loss: 2.9672
Epoch 858/1000
325/325 [==============================] - 0s 273us/step - loss: 2.9741
Epoch 859/1000
325/325 [==============================] - 0s 273us/step - loss: 2.9765
Epoch 860/1000
325/325 [==============================] - 

In [ ]:
# pred_B = val_B[1]
# pred1_B = np.round(pred_B)
# pred1_B


In [ ]:
# z_B=y_t_B
# r = math.sqrt(mean_squared_error(z_B,pred1_B))
# r
# print(np.exp(-r))

In [ ]:
x_C

In [30]:
val_C=train_pred_eval_model(x_C,y_C,x_C,y_C,5,0.5,'adam',1000,10)


Exception ignored in: <function EagerResourceDeleter.__del__ at 0x7fd01b453598>
Traceback (most recent call last):
  File "/home/shobhit/Documents/interiit/env/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py", line 263, in __del__
    def __del__(self):
KeyboardInterrupt: 


Epoch 1/1000
325/325 [==============================] - 122s 375ms/step - loss: 9.4312
Epoch 2/1000
325/325 [==============================] - 0s 230us/step - loss: 8.7285
Epoch 3/1000
 10/325 [..............................] - ETA: 0s - loss: 12.0707

/home/shobhit/Documents/interiit/env/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (2.483362). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


325/325 [==============================] - 0s 230us/step - loss: 7.8986
Epoch 4/1000
325/325 [==============================] - 0s 257us/step - loss: 7.5007
Epoch 5/1000
325/325 [==============================] - 0s 254us/step - loss: 6.6395
Epoch 6/1000
325/325 [==============================] - 0s 256us/step - loss: 5.8824
Epoch 7/1000
325/325 [==============================] - 0s 256us/step - loss: 5.4624
Epoch 8/1000
325/325 [==============================] - 0s 263us/step - loss: 5.1835
Epoch 9/1000
325/325 [==============================] - 0s 256us/step - loss: 4.5697
Epoch 10/1000
325/325 [==============================] - 0s 260us/step - loss: 4.2350
Epoch 11/1000
325/325 [==============================] - 0s 253us/step - loss: 4.0320
Epoch 12/1000
325/325 [==============================] - 0s 254us/step - loss: 4.4883
Epoch 13/1000
325/325 [==============================] - 0s 254us/step - loss: 4.0570
Epoch 14/1000
325/325 [==============================] - 0s 258us/step - l

325/325 [==============================] - 0s 260us/step - loss: 3.0659
Epoch 193/1000
325/325 [==============================] - 0s 253us/step - loss: 3.0902
Epoch 194/1000
325/325 [==============================] - 0s 253us/step - loss: 3.0803
Epoch 195/1000
325/325 [==============================] - 0s 253us/step - loss: 3.1057
Epoch 196/1000
325/325 [==============================] - 0s 254us/step - loss: 3.0558
Epoch 197/1000
325/325 [==============================] - 0s 255us/step - loss: 3.0221
Epoch 198/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9927
Epoch 199/1000
325/325 [==============================] - 0s 257us/step - loss: 3.2162
Epoch 200/1000
325/325 [==============================] - 0s 249us/step - loss: 2.9712
Epoch 201/1000
325/325 [==============================] - 0s 259us/step - loss: 3.0148
Epoch 202/1000
325/325 [==============================] - 0s 254us/step - loss: 3.0275
Epoch 203/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 253us/step - loss: 3.0494
Epoch 381/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9838
Epoch 382/1000
325/325 [==============================] - 0s 253us/step - loss: 3.0312
Epoch 383/1000
325/325 [==============================] - 0s 250us/step - loss: 2.9545
Epoch 384/1000
325/325 [==============================] - 0s 258us/step - loss: 2.9626
Epoch 385/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9624
Epoch 386/1000
325/325 [==============================] - 0s 255us/step - loss: 2.9824
Epoch 387/1000
325/325 [==============================] - 0s 251us/step - loss: 3.0031
Epoch 388/1000
325/325 [==============================] - 0s 252us/step - loss: 2.9598
Epoch 389/1000
325/325 [==============================] - 0s 255us/step - loss: 2.9261
Epoch 390/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9755
Epoch 391/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 258us/step - loss: 3.0452
Epoch 475/1000
325/325 [==============================] - 0s 261us/step - loss: 2.9642
Epoch 476/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9845
Epoch 477/1000
325/325 [==============================] - 0s 261us/step - loss: 2.9800
Epoch 478/1000
325/325 [==============================] - 0s 257us/step - loss: 3.0082
Epoch 479/1000
325/325 [==============================] - 0s 265us/step - loss: 2.9517
Epoch 480/1000
325/325 [==============================] - 0s 258us/step - loss: 2.9975
Epoch 481/1000
325/325 [==============================] - 0s 255us/step - loss: 2.9750
Epoch 482/1000
325/325 [==============================] - 0s 258us/step - loss: 2.9597
Epoch 483/1000
325/325 [==============================] - 0s 258us/step - loss: 2.9505
Epoch 484/1000
325/325 [==============================] - 0s 258us/step - loss: 3.0014
Epoch 485/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 252us/step - loss: 2.9828
Epoch 663/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9757
Epoch 664/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9888
Epoch 665/1000
325/325 [==============================] - 0s 256us/step - loss: 2.9722
Epoch 666/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9828
Epoch 667/1000
325/325 [==============================] - 0s 251us/step - loss: 2.9747
Epoch 668/1000
325/325 [==============================] - 0s 257us/step - loss: 2.9731
Epoch 669/1000
325/325 [==============================] - 0s 259us/step - loss: 2.9578
Epoch 670/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9867
Epoch 671/1000
325/325 [==============================] - 0s 256us/step - loss: 2.9727
Epoch 672/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9861
Epoch 673/1000
325/325 [==============================] - 

325/325 [==============================] - 0s 251us/step - loss: 2.9548
Epoch 851/1000
325/325 [==============================] - 0s 249us/step - loss: 2.9588
Epoch 852/1000
325/325 [==============================] - 0s 254us/step - loss: 2.9614
Epoch 853/1000
325/325 [==============================] - 0s 258us/step - loss: 2.9476
Epoch 854/1000
325/325 [==============================] - 0s 259us/step - loss: 2.9658
Epoch 855/1000
325/325 [==============================] - 0s 257us/step - loss: 2.9729
Epoch 856/1000
325/325 [==============================] - 0s 265us/step - loss: 2.9649
Epoch 857/1000
325/325 [==============================] - 0s 262us/step - loss: 2.9637
Epoch 858/1000
325/325 [==============================] - 0s 256us/step - loss: 2.9626
Epoch 859/1000
325/325 [==============================] - 0s 257us/step - loss: 2.9726
Epoch 860/1000
325/325 [==============================] - 0s 253us/step - loss: 2.9601
Epoch 861/1000
325/325 [==============================] - 

In [ ]:
# pred_C = val_C[1]
# pred1_C = np.round(pred_C)
# pred1_C


In [ ]:
# z_C=y_t_C
# r = math.sqrt(mean_squared_error(z_C,pred1_C))
# print(r)
# print(np.exp(-r))

In [ ]:
model_A=val_A
model_B=val_B
model_C=val_C

In [ ]:
test.drop(['Date'],inplace=True,axis=1)


In [ ]:
print(test)

In [ ]:
test=np.array(test)
test

In [ ]:
test

In [ ]:
test = np.reshape(test, (test.shape[0], 1, test.shape[1]))


In [ ]:
# A=model_A.predict(test)
A=(model_A.predict(test))
B=(model_B.predict(test))
C=(model_C.predict(test))
# print(A)

In [ ]:
final=[]
for i in range(len(A)):
    final.append(A[i])
    final.append(B[i])
    final.append(C[i])
df=pd.DataFrame(final,columns = ['n_Procedure'])
    

In [ ]:
df

In [ ]:
df.to_csv('Final.csv')